# Create Innovate UK Awards

Creates awards from Innovate UK funded projects database. ~41K projects.

**Prerequisites:**
- Run `scripts/local/innovateuk_to_s3.py` to process and upload the data first.

**Data source:** https://www.ukri.org/publications/innovate-uk-funded-projects-since-2004/  
**S3 location:** `s3a://openalex-ingest/awards/innovateuk/innovateuk_projects.parquet`

**Innovate UK funder:**
- funder_id: 4320335087
- display_name: "Innovate UK"
- ROR: https://ror.org/05ar5fy68
- DOI: 10.13039/501100006041

**Notes:**
- Innovate UK grants are to **organizations**, primarily businesses (SMEs, startups)
- The grantee is typically a company, not a PI
- Data is filtered to lead participants only (one record per project)
- product_type maps to funder_scheme (e.g., "Collaborative R&D", "Feasibility Studies")
- sector provides additional categorization (e.g., "AI & Data Economy", "Manufacturing")

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.innovateuk_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/innovateuk/innovateuk_projects.parquet`;

In [ ]:
%sql
-- Check row count (should be ~41K)
SELECT COUNT(*) as total_projects FROM openalex.awards.innovateuk_raw;

In [ ]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.innovateuk_raw LIMIT 5;

## Step 2: Create Innovate UK Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.innovateuk_awards
USING delta
AS
WITH
-- Get Innovate UK funder from OpenAlex by explicit funder_id
innovateuk_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320335087  -- Innovate UK
),

awards_transformed AS (
    SELECT
        -- Generate unique ID: xxhash64 of funder_id:project_number
        abs(xxhash64(CONCAT(f.funder_id, ':', CAST(p.project_number AS STRING)))) % 9000000000 as id,

        -- Display name = project_title
        p.project_title as display_name,

        -- Description = public_description
        p.public_description as description,

        -- Funder info
        f.funder_id,
        CAST(p.project_number AS STRING) as funder_award_id,

        -- Amount in GBP
        CAST(p.award_amount AS DOUBLE) as amount,
        'GBP' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - map product_type to standardized types
        CASE
            WHEN LOWER(p.product_type) LIKE '%fellowship%' THEN 'fellowship'
            WHEN LOWER(p.product_type) LIKE '%feasibility%' THEN 'research'
            WHEN LOWER(p.product_type) LIKE '%r&d%' OR LOWER(p.product_type) LIKE '%research%' THEN 'research'
            WHEN LOWER(p.product_type) LIKE '%innovation%' THEN 'research'
            WHEN LOWER(p.product_type) LIKE '%demonstrator%' THEN 'research'
            WHEN LOWER(p.product_type) LIKE '%smart%' THEN 'research'
            WHEN LOWER(p.product_type) LIKE '%knowledge transfer%' THEN 'training'
            WHEN LOWER(p.product_type) LIKE '%loan%' THEN 'loan'
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = product_type (e.g., "Collaborative R&D", "Feasibility Studies")
        p.product_type as funder_scheme,

        -- Provenance
        'innovate_uk' as provenance,

        -- Dates (stored as strings in YYYY-MM-DD format)
        TRY_TO_DATE(p.start_date, 'yyyy-MM-dd') as start_date,
        TRY_TO_DATE(p.end_date, 'yyyy-MM-dd') as end_date,
        YEAR(TRY_TO_DATE(p.start_date, 'yyyy-MM-dd')) as start_year,
        YEAR(TRY_TO_DATE(p.end_date, 'yyyy-MM-dd')) as end_year,

        -- Lead investigator - Innovate UK grants go to organizations, not individuals
        -- We store the lead participant organization in the affiliation field
        CASE
            WHEN p.participant_name IS NOT NULL THEN
                struct(
                    CAST(NULL AS STRING) as given_name,
                    CAST(NULL AS STRING) as family_name,
                    CAST(NULL AS STRING) as orcid,
                    CAST(NULL AS DATE) as role_start,
                    struct(
                        p.participant_name as name,
                        'United Kingdom' as country,  -- All Innovate UK grants are UK-based
                        CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not available)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL - Gateway to Research project page
        CONCAT('https://gtr.ukri.org/projects?ref=', CAST(p.project_number AS STRING)) as landing_page_url,

        -- No DOI for Innovate UK grants
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', CAST(p.project_number AS STRING)))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date,

        -- Additional metadata for analysis
        p.sector,
        p.enterprise_size,
        p.region,
        p.competition_year,
        p.project_status,
        p.is_iscf

    FROM openalex.awards.innovateuk_raw p
    CROSS JOIN innovateuk_funder f
    WHERE p.project_number IS NOT NULL
)

SELECT * FROM awards_transformed;

In [ ]:
%sql
-- Remove previous data for this source before inserting fresh data
DELETE FROM openalex.awards.openalex_awards_raw
WHERE provenance = 'innovate_uk';

-- Insert into openalex_awards_raw with priority 28 (after European Commission at 27)
INSERT INTO openalex.awards.openalex_awards_raw
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    28 as priority
FROM openalex.awards.innovateuk_awards;

## Verification Queries

In [ ]:
%sql
-- Check row count (should be ~41K)
SELECT COUNT(*) as total_innovateuk_awards FROM openalex.awards.innovateuk_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    amount,
    currency,
    start_date,
    end_date,
    lead_investigator.affiliation.name as grantee,
    sector,
    enterprise_size
FROM openalex.awards.innovateuk_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be Innovate UK)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.innovateuk_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funding_type distribution
SELECT funding_type, COUNT(*) as cnt
FROM openalex.awards.innovateuk_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme distribution (product types)
SELECT funder_scheme, COUNT(*) as cnt, 
       ROUND(SUM(amount)/1e9, 2) as total_amount_billions_gbp
FROM openalex.awards.innovateuk_awards
WHERE funder_scheme IS NOT NULL
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check sector distribution
SELECT sector, COUNT(*) as cnt, 
       ROUND(SUM(amount)/1e9, 2) as total_amount_billions_gbp
FROM openalex.awards.innovateuk_awards
WHERE sector IS NOT NULL
GROUP BY sector
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check enterprise size distribution
SELECT enterprise_size, COUNT(*) as cnt, 
       ROUND(SUM(amount)/1e9, 2) as total_amount_billions_gbp
FROM openalex.awards.innovateuk_awards
WHERE enterprise_size IS NOT NULL
GROUP BY enterprise_size
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_description,
    COUNT(amount) as has_amount,
    COUNT(start_date) as has_start_date,
    COUNT(end_date) as has_end_date,
    COUNT(lead_investigator) as has_grantee_info,
    ROUND(COUNT(display_name) * 100.0 / COUNT(*), 1) as pct_title,
    ROUND(COUNT(amount) * 100.0 / COUNT(*), 1) as pct_amount,
    ROUND(SUM(amount)/1e9, 2) as total_amount_billions_gbp
FROM openalex.awards.innovateuk_awards;

In [ ]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt,
       ROUND(SUM(amount)/1e9, 2) as total_amount_billions_gbp
FROM openalex.awards.innovateuk_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 25;

In [ ]:
%sql
-- Top grantee organizations
SELECT 
    lead_investigator.affiliation.name as grantee,
    COUNT(*) as grant_count,
    ROUND(SUM(amount)/1e6, 1) as total_amount_millions_gbp
FROM openalex.awards.innovateuk_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY total_amount_millions_gbp DESC
LIMIT 25;

In [ ]:
%sql
-- Check region distribution
SELECT 
    region,
    COUNT(*) as grant_count,
    ROUND(SUM(amount)/1e9, 2) as total_amount_billions_gbp
FROM openalex.awards.innovateuk_awards
WHERE region IS NOT NULL
GROUP BY region
ORDER BY total_amount_billions_gbp DESC;

In [ ]:
%sql
-- ISCF (Industrial Strategy Challenge Fund) breakdown
SELECT 
    is_iscf,
    COUNT(*) as grant_count,
    ROUND(SUM(amount)/1e9, 2) as total_amount_billions_gbp
FROM openalex.awards.innovateuk_awards
GROUP BY is_iscf;